In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip install transformers
!pip install bert-extractive-summarizer

     |████████████████████████████████| 2.0MB 8.0MB/s 
     |████████████████████████████████| 890kB 51.3MB/s 
     |████████████████████████████████| 3.2MB 53.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=772c6dbecdf9786fe3beaef0e2fa21970d3d6563b43ddb9d9cad0ed34b5d4f4e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from transformers import BertTokenizer, TFBertModel

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AY0e-g7NAQUAF55d5insvWjm22gvHPuzbhHCIvut1_0i4OcBmeXvc5ySKZA
Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/newsdf.csv"
df = pd.read_csv(path)

In [ ]:
df.head()

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [ ]:
texts = df['text']
texts_list = list(texts)
filtered_texts = []
for text in texts_list:
    if (isinstance(text, str)):
        filtered_texts.append(text)

In [ ]:
all(isinstance(x, str) for x in filtered_texts)

True

In [ ]:
from summarizer import Summarizer
model = Summarizer()

In [ ]:
series = list(df['text'])
series = series[:100]
results = pd.DataFrame(series)
results

,0
0,March 27(Reuters) - AU Optronics Corp :\n* Say...
1,"March 8, 2018 / 1:35 PM / Updated an hour ago ..."
2,"Dropbox shares surge in IPO Saturday, March 24..."
3,BERLIN (Reuters) - The man known as the “bookk...
4,Dow closes 336 points higher as trade-war worr...
...,...
95,/ ivory coast Amazon shares fall after report ...
96,"March 6, 2018 / 4:25 PM / Updated an hour ago ..."
97,The number in the corner of Upton Martin’s Med...
98,March 8 (Reuters) - Post Holdings Inc:\n* POST...


In [ ]:
summaries = []
for t in list(results[0]):
  result = model(t)
  summaries.append(''.join(result))

In [ ]:
summaries[0]

'March 27(Reuters) - AU Optronics Corp :\n* Says it plans to pay cash dividend of T$1.2/share for 2017\nSource text in Chinese: goo.gl/uxuxci\nFurther company coverage: (Beijing Headline News)'

In [ ]:
results['summaries'] = pd.Series(summaries)

In [ ]:
np.mean(np.array(results[0]) == np.array(results['summaries']))

0.12

About 12% of the summarized articles are identicle to the original article text.

**Now that we have summarized the first 100 articles in our corpus, let us use Rouge metric to attemp to score our initial attemp.** We use PyRouge, a summarization evaluation package to do so.

In [ ]:
!pip install rouge-metric

In [ ]:
from rouge_metric import PyRouge
rouge = PyRouge(rouge_n=(1, 2, 4), rouge_l=True, rouge_w=True,
                rouge_w_weight=1.2, rouge_s=True, rouge_su=True, skip_gap=4)

In [ ]:
hypotheses = list(results['summaries'])
references = list(results[0])

In [ ]:
scores = rouge.evaluate(hypotheses, references)

In [ ]:
print(scores)

{'rouge-1': {'r': 0.06523690766392178, 'p': 0.0007186859468884007, 'f': 0.001421709553041831}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-4': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.06523690766392178, 'p': 0.0007186859468884007, 'f': 0.001421709553041831}, 'rouge-w-1.2': {'r': 0.09970745104256099, 'p': 0.0011484129404315737, 'f': 0.0022706726711306307}, 'rouge-s4': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-su4': {'r': 0.0, 'p': 0.0, 'f': 0.0}}


We get a Rouge score of 0.065/1. This isn't great . **Next, we attemp to summarize a document by simply using the first 3 sentences.**

In [ ]:
import re
sentences = list(results['summaries'])
first_3 = []
for sentence in sentences:
  first_3.append(' '.join(re.split(r'(?<=[.:;])\s', sentence)[:4]))
results['first_3'] = pd.Series(first_3)

In [ ]:
hypotheses_first_3 = list(results['first_3'])
scores = rouge.evaluate(hypotheses_first_3, references)
print(scores)

{'rouge-1': {'r': 0.06155785438684868, 'p': 0.0007677222660188229, 'f': 0.001516531029447086}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-4': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.06155785438684868, 'p': 0.0007677222660188229, 'f': 0.001516531029447086}, 'rouge-w-1.2': {'r': 0.09425793349131079, 'p': 0.0012270672139771499, 'f': 0.00242259661706262}, 'rouge-s4': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-su4': {'r': 0.0, 'p': 0.0, 'f': 0.0}}


Using this method, we get a Rouge score of 0.062/1 . It turns out that using the first 3 sentences instead of the initial Bert summarizer does not change our accuracy by much.

In [ ]:
bert_inputs = tokenizer(filtered_texts,
                  padding=True,
                  return_tensors='tf')

bert_inputs

Token indices sequence length is longer than the specified maximum sequence length for this model (892 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
bert_layer = TFBertModel.from_pretrained('bert-base-cased')